In [1]:
import tensorflow as tf

In [2]:
@tf.function
def extract_matrix(f):
    lines = tf.strings.split(f, '\n')
    lines = tf.strings.split(lines, ' bags contain ')
    lines = lines.to_tensor()
    bagtypes = lines[:,0]
    n = tf.shape(bagtypes)[0]
    
    init = tf.lookup.KeyValueTensorInitializer(bagtypes, tf.range(n))
    index_map = tf.lookup.StaticHashTable(init, default_value=-1)
    sg_idx = index_map[tf.constant('shiny gold')]
    
    indices = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
    updates = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    
    ta_i = 0

    for i in tf.range(n):
        c = lines[i][1]
        c = tf.strings.split(c, ', ')
        for j in tf.range(tf.shape(c)[0]):
            e = c[j]
            e_split = tf.strings.split(e, ' ')
            bagtype = e_split[1] + tf.constant(' ') + e_split[2]
            k = index_map[bagtype]

            if k > -1:
                amt = tf.strings.to_number(e_split[0])
                indices = indices.write(ta_i, (i, k))
                updates = updates.write(ta_i, amt)
                ta_i += 1

    return tf.scatter_nd(indices.stack(), updates.stack(), (n, n)), sg_idx

@tf.function(experimental_compile=True)
def solve(X, sg_idx, part=1):
    X_orig = X
    res = tf.zeros_like(X)    
    while tf.math.count_nonzero(X) > 0:
        res += X
        X = X@X_orig
    if part == 1:
        return tf.math.count_nonzero(res[:,sg_idx])
    elif part == 2:
        return tf.reduce_sum(res[sg_idx])

In [3]:
#%%time
X, sg_idx = extract_matrix(tf.io.read_file('day7.txt'))

In [4]:
%%timeit
solve(X, sg_idx, part=1)

2.86 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
solve(X, sg_idx, part=2)

2.28 ms ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
